In [1]:
%cd image_retrieval/


/home/anlab/Downloads/imageclassification/image_retrieval


In [2]:
path_local="/home/anlab/Downloads/imageclassification/"

***Lấy file query ***

In [3]:

import os
def get_file_query():
  path= "gt_files/"
  list_gt = os.listdir(path)
  list_files_query=[]
  for file_name in list_gt:
    name=file_name.split(".")[0].split("_")[-1]
    if(name=="query"):
      list_files_query.append(file_name)
  return list_files_query



**lấy file ảnh tương ứng với querry (gồm 3 file good, oke, junk)**

In [4]:
#get file good,oke,junk correspond
def get_file_name(list_files_query, name):
  list_files_name=[]
  for query_file in list_files_query:
    name_correspond= query_file.replace("query",name)
    list_files_name.append(name_correspond)
  return list_files_name
  





**lấy ảnh trong file query**

In [5]:
def get_image_query(list_files_query):
  list_image_query=[]
  for query in list_files_query:
    dir= path_local+"/image_retrieval/gt_files/"+query
    with open(dir,"r") as f:
      for line in f.readlines():
        name_image= line.split(" ")[0].split("_",1)[1]+".jpg"
        list_image_query.append(name_image)
  return list_image_query

**lấy ảnh tương ứng của query với từng file good, oke, junk**

In [6]:


def get_image_correspond(file_name):
  list_image_name= []
  with open("gt_files/"+file_name,"r") as f:
    for line in f.readlines():
      list_image_name.append(line.split("\n")[0]+".jpg")
  return list_image_name

#Rmac


In [7]:
%cd keras_rmac/


/home/anlab/Downloads/imageclassification/image_retrieval/keras_rmac


In [8]:
from __future__ import division
from __future__ import print_function
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.layers import Lambda, Dense, TimeDistributed, Input
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
import keras
# from vgg16 import VGG16
# from RoiPooling import RoiPooling
from PIL import Image,ImageOps
import scipy.io
import numpy as np
import utils
from torchvision import models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch
import cv2

def rzac(x, L=3, eps=1e-6):
#     N, C, H, W = x.size()
    feature_total= []
    ovr = 0.4 # desired overlap of neighboring regions
    steps = torch.Tensor([2, 3, 4, 5, 6, 7]) # possible regions for the long dimension

    W = x.size(3)
    H = x.size(2)

    w = min(W, H)
    w2 = math.floor(w/2.0 - 1)

    b = (max(H, W)-w)/(steps-1)
    (tmp, idx) = torch.min(torch.abs(((w**2 - w*b)/w**2)-ovr), 0) # steps(idx) regions for long dimension

    # region overplus per dimension
    Wd = 0;
    Hd = 0;
    if H < W:  
        Wd = idx.item() + 1
    elif H > W:
        Hd = idx.item() + 1

    v = F.max_pool2d(x, (x.size(-2), x.size(-1)))
    v = v / (torch.norm(v, p=2, dim=1, keepdim=True) + eps).expand_as(v)
    
    for l in range(1, L+1):
        
        wl = math.floor(2*w/(l+1))
        wl2 = math.floor(wl/2 - 1)
        
        if l+Wd == 1:
            b = 0
        else:
            b = (W-wl)/(l+Wd-1)
        cenW = torch.floor(wl2 + torch.Tensor(range(l-1+Wd+1))*b) - wl2 # center coordinates
        if l+Hd == 1:
            b = 0
        else:
            b = (H-wl)/(l+Hd-1)
        cenH = torch.floor(wl2 + torch.Tensor(range(l-1+Hd+1))*b) - wl2 # center coordinates
        
        for i_ in cenH.tolist():
            for j_ in cenW.tolist():
                if wl == 0:
                    continue
                R = x[:,:,(int(i_)+torch.Tensor(range(wl)).long()).tolist(),:]
                R = R[:,:,:,(int(j_)+torch.Tensor(range(wl)).long()).tolist()]
                vt = F.max_pool2d(R, (R.size(-2), R.size(-1)))
                vt = vt / (torch.norm(vt, p=2, dim=1, keepdim=True) + eps).expand_as(vt)
#                 v += vt
                feature_total.append(vt)
    feature_total=np.vstack(feature_total)
    feature_total=np.squeeze(feature_total,axis=2)
    feature_total= np.transpose(feature_total,(2,0,1))
    feature_total=torch.from_numpy(feature_total)
    return feature_total


Using TensorFlow backend.


In [9]:
def get_feature(reg_feat_mat):
  reg_feat_mat = reg_feat_mat.squeeze() # [1, r, c] -> [r, c]
#   r = reg_feat_mat.size(0)
  ag_feat_vec = torch.sum(reg_feat_mat, dim=0, keepdim=False) # (r, c) -> (c,)
  ag_feat_vec = ag_feat_vec / (torch.norm(ag_feat_vec,
                                            p=2,
                                            dim=0,
                                            keepdim=True) + 1e-6)
  ag_feat_vec=np.array(ag_feat_vec).reshape(1,-1)
  return ag_feat_vec

In [10]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tqdm
def load_image_and_bbs(im_path, bb_path):
    img = cv2.imread(im_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    bb_df = pd.read_csv(bb_path, sep=' ', header=None, index_col=False)
    bb_mat = bb_df.to_numpy()
    return img, bb_mat

def _get_masked_img( img, bbs):
    patches = []
#     masked = np.zeros_like(img)
    for bb in bbs:

        x_l = bb[0]
        x_r = bb[0] + bb[2]
        y_u = bb[1]
        y_d = bb[1] + bb[3]
        patches.append(img[y_u:y_d, x_l:x_r])
#         masked[y_u:y_d, x_l:x_r] = img[y_u:y_d, x_l:x_r]

    return patches

In [11]:
import os
list_files_query=os.listdir(path_local+"data/pg_data/query")
from shutil import copyfile
path=path_local+"data/pg_data/"
for i in range(len(list_files_query)):
  name_file=list_files_query[i]
  with open(path+"query/"+name_file,"r") as f:
    for line in f.readlines():
      line= line.split(" ",1)
      name_images=line[0].split("_",1)[1]+".jpg"
      coordinate= line[1].split("\n")[0].split(" ")
      coordinate=[round(float(value)) for value in coordinate]
      coordinate=str(coordinate[0])+" "+str(coordinate[1])+" "+str(coordinate[2]-coordinate[0])+" "+str(coordinate[3]-coordinate[1])
      if(i+1<10):
        copyfile(path+"Images/"+name_images, path+"Queries/"+f"0{i+1}.jpg")
        with open(path+"Queries/"+f"0{i+1}.txt","w") as f:
          f.write(coordinate)
      else :
        copyfile(path+"Images/"+name_images, path+"Queries/"+f"{i+1}.jpg")
        with open(path+"Queries/"+f"{i+1}.txt","w") as f:
          f.write(coordinate)

In [12]:
import os
query_dir = path_local+'data/pg_data/Queries'
im_bb_path_tuples = [(os.path.join(query_dir, '{:02d}.jpg'.format(i)),
                      os.path.join(query_dir, '{:02d}.txt'.format(i))
                     ) for i in range(1, 56)
                    ]
len(im_bb_path_tuples)

55

In [18]:
def mac(x):
    vt= F.max_pool2d(x, (x.size(-2), x.size(-1)))
    vt=np.squeeze(vt,axis=2)
    vt= np.transpose(vt,(2,0,1))
    return vt

In [19]:
#query mac

import math
import keras
from tqdm import tqdm
model= keras.applications.VGG16(include_top=False,weights=utils.DATA_DIR + utils.WEIGHTS_FILE)
vector_query =[]
for value in tqdm(im_bb_path_tuples):
    demo_im_path, demo_bb_path = im_bb_path_tuples[0]
    demo_im, demo_bb = load_image_and_bbs(demo_im_path, demo_bb_path)
    masked=_get_masked_img(demo_im, demo_bb)
    img=masked[0]
    x = change_chanel(img)
    im_tensor = model.predict(x)
    im_tensor=torch.tensor(im_tensor)
    reg_feat_mat= mac(im_tensor)
    MAC=reg_feat_mat = reg_feat_mat.squeeze(0) # [1, r, c] -> [r, c]
    vector_query.append(MAC[-1])
print(len(vector_query))

100%|██████████| 55/55 [00:03<00:00, 14.45it/s]

55


In [16]:
#query rmac

# from tqdm import tqdm
# vector_query =[]

# for value in tqdm(im_bb_path_tuples):
    
#     demo_im_path, demo_bb_path = value
#     demo_im, demo_bb = load_image_and_bbs(demo_im_path, demo_bb_path)
#     masked=_get_masked_img(demo_im, demo_bb)
#     x= np.moveaxis(masked[0], -1, 0)
#     x=np.expand_dims(x, axis=0)
#     im_tensor=model.predict(x)
#     im_tensor=torch.tensor(im_tensor)
#     reg_feat_mat= rzac(im_tensor)
# #     print("reg_feat_mat",reg_feat_mat.shape)
#     RMAC=get_feature(reg_feat_mat)
#     vector_query.append(RMAC[-1])
    
# print(len(vector_query))

In [20]:
def get_feature_Db(reg_feat_mat):
    reg_feat_mat = reg_feat_mat.squeeze() # [1, r, c] -> [r, c]
    return reg_feat_mat

In [21]:
def change_chanel(tensor_img):
    x= np.moveaxis(tensor_img, -1, 0)
    x=np.expand_dims(x, axis=0)
    return x

In [22]:
def preprocessImage(linkImage):
    img=cv2.imread(linkImage)
#     img=cv2.resize(img,(724,724))
    x = change_chanel(img)
    return x

In [23]:
def get_Rmac(file,model):
    x = preprocessImage(file)
    #get feature each tensor image
    im_tensor = model.predict(x)
    #conver to torch tensor
    im_tensor=torch.tensor(im_tensor)
    #caculate rzac
    reg_feat_mat= rzac(im_tensor)   
    #caculate rmac 
    RMAC = get_feature(reg_feat_mat)
    return RMAC


In [24]:
def get_Rmac_DB(file,model):
    x = preprocessImage(file)
    #get feature each tensor image
    im_tensor = model.predict(x)
    #conver to torch tensor
    im_tensor=torch.tensor(im_tensor)
    #caculate rzac
    reg_feat_mat= rzac(im_tensor)
    nVector_represent=get_feature_Db(reg_feat_mat)
    return nVector_represent
    

In [25]:
%cd ..

/home/anlab/Downloads/imageclassification/image_retrieval


In [28]:
%cd keras_rmac

/home/anlab/Downloads/imageclassification/image_retrieval/keras_rmac


In [29]:
# get file image correspond with query
im_list_query=get_image_query(list_files_query)
print(len(im_list_query))

55


In [30]:
import pickle
with open('dataset.pickle',"rb") as f:
    dataset = pickle.load(f)

In [33]:
import pickle
from tqdm import tqdm
with open('dataset.pickle',"rb") as f:
    dataset = pickle.load(f)
dataset_new=[]
vector_dataset=[]
for img in tqdm(dataset):
    file = path_local+"Object Dataset/test/oxbuild_images/"+img
    # Load RMAC model
    RMAC_DB=get_Rmac_DB(file,model)
    vector_dataset.append(RMAC_DB)
    for i in range(RMAC_DB.shape[0]):
        dataset_new.append(img)

100%|██████████| 5063/5063 [20:00<00:00,  4.22it/s] 


In [34]:
# with open('dataset_new.pickle', 'wb') as f:
#     pickle.dump(dataset_new, f)
# with open('vector_dataset.pickle', 'wb') as f:
#     pickle.dump(vector_dataset, f)

/home/anlab/anaconda3/envs/myenvs/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [35]:
with open('dataset.pickle',"rb") as f:
    dataset = pickle.load(f)
with open('dataset_new.pickle',"rb") as f:
    dataset_new = pickle.load(f)
with open('vector_dataset.pickle',"rb") as f:
    vector_dataset = pickle.load(f)

In [33]:
len(dataset_new)

102106

In [36]:
vector_dataset=np.vstack(vector_dataset)
print(vector_dataset.shape)


(102106, 512)


In [37]:
vector_query_test=[t.numpy() for t in vector_query]
vector_query=np.array(vector_query_test)
print(vector_query.shape)


(55, 512)


In [38]:
# tính giao của 2 array-> kết quả trả về theo thứ tự của array 1
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [39]:
%cd /home/anlab/Downloads/imageclassification/image_retrieval/

/home/anlab/Downloads/imageclassification/image_retrieval


In [40]:
# %cd ..

In [41]:
list_files_good=get_file_name(list_files_query,"good")

In [42]:
topk=len(dataset_new)
topk

102106

In [43]:
from collections import OrderedDict
def result_image_query(vector_query_x, topk):
    
    #distance query_x with datasets
    vector_l2_dataset= np.linalg.norm(vector_query_x- vector_dataset ,axis=-1)
    
    #sort index image
    toplen_image_query= np.argsort(vector_l2_dataset)
    
    image_correspond = [dataset_new[image] for image in toplen_image_query]
    image_correspond = list(OrderedDict.fromkeys(image_correspond))
    image_topk= image_correspond[:topk]
    
    return image_topk
    

In [44]:
#test search image 
import math
# model= keras.applications.VGG16(include_top=False,weights=utils.DATA_DIR + utils.WEIGHTS_FILE)
file = path_local+"Object Dataset/test/oxbuild_images/"+"ashmolean_000106.jpg"
x = preprocessImage(file)
im_tensor = model.predict(x)
im_tensor=torch.tensor(im_tensor)
feature=rzac(im_tensor)
rmac_file_test=get_feature(feature)


image_topk= result_image_query(rmac_file_test,5)
image_topk

['ashmolean_000106.jpg',
 'ashmolean_000000.jpg',
 'ashmolean_000028.jpg',
 'oxford_000904.jpg',
 'oxford_000575.jpg']

In [45]:
from collections import OrderedDict
def map_name(list_files_query,name,vector_query,vector_dataset,dataset_new,topk):
  list_ap_name=[]
  list_files_name=get_file_name(list_files_query,name)
    
  for i in range(len(list_files_query)):
    #get grount truth
    gt=get_image_correspond(list_files_name[i])
    
    image_correspond = result_image_query(vector_query[i],topk)
    index=[]
    AP=0
    for j in range(len(image_correspond)):
      if(image_correspond[j] in gt):
        index.append(j)
        
    for k in range(len(index)):
      AP= AP+ ((k+1)/(index[k]+1))
    AP=AP/len(gt)
    list_ap_name.append(AP)
    
  map_name=sum(list_ap_name)/len(list_ap_name)

  return map_name

In [46]:
map_good= map_name(list_files_query,"good",vector_query,vector_dataset,dataset_new,topk)

In [47]:
print(map_good)


0.07715097784412402
